## Imports

In [146]:
import numpy as np
from scipy import optimize as opt
from scipy.sparse import linalg

## Data analysis

Caricamento della matrice $X$ da file.

In [17]:
fname = 'dataset/ML-CUP19-TR.csv'
X     = np.loadtxt(fname,delimiter=',')
X     = X[:,1:-2] # Remove ID and target
n, k  = X.shape
(n, k)

(1765, 20)

Costruzione della matrice $\hat{X}$ come indicato dal progetto.

In [24]:
I  = np.eye(n)
XH = np.concatenate((X.T, I), axis=0)
m, n = XH.shape
(m, n)

(1785, 1765)

Costruzione della Hessiana, conferma che $\nabla^2 f(x) = \hat{X}^T\hat{X} = X X^T+I$ e dunque positive definite.

In [37]:
assert (XH.T @ XH).all() == (X@X.T + I).all()
H   = X@X.T + I
eig = np.linalg.eigvals(H)
np.all(eig > 0)

True

$y$ è un vettore casuale.

In [255]:
sigma = 50
mu    = 15
y = np.random.random(m) * sigma + mu
y.shape

(1785,)

## Direct Numpy solution

In [256]:
x_1 = np.linalg.pinv(XH) @ y
x_1.shape

(1765,)

In [257]:
x_2 = np.linalg.lstsq(XH, y, rcond=None)[0]
x_2.shape

(1765,)

In [258]:
np.linalg.norm(x_1-x_2)

3.2789388262071844e-12

## Newton method

In [259]:
f   = lambda w: 1/2 * np.linalg.norm(XH@w - y)**2
f_1 = lambda w: XH.T @ (XH@w - y)
f_2 = lambda w: XH.T @ XH

In [260]:
start = np.random.random(n)
x_3 = opt.minimize(f, start, 
                   #method='L-BFGS-B', 
                   jac=f_1,
                   hess=f_2,
                   callback=lambda x: print(f(x)))

/home/rmassidda/envs/notebook/lib64/python3.8/site-packages/scipy/optimize/_minimize.py:522: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


1923578.112505273
1833005.0142160323
1812425.1891567379
1806210.4754677257
1798065.8457326954
1723787.7265559856
1582590.2545861865
1494965.0179496915
1488036.838035074
1431847.205251294
1324534.493537734
1292349.823143461
1027143.1970579764
851028.6502295064
851028.6502294769


In [262]:
def newton(f, start, fprime, fsecond, maxiter=30):
    x = start
    print(x, f(x))
    for _ in range(maxiter):
        d = - np.linalg.inv(fsecond(x)) @ fprime(x)
        x = x + d
        print(x, f(x))
    return x

In [263]:
x_4 = newton(f, start, f_1, f_2, 5)

[0.34942811 0.70831026 0.86268772 ... 0.5199621  0.45611373 0.86325698] 1988312.3914933947
[ 22.91079788  46.77163103 -11.24995538 ...   3.94064607  22.42362675
  60.03761232] 851028.6502294771
[ 22.91079788  46.77163103 -11.24995538 ...   3.94064607  22.42362675
  60.03761232] 851028.6502294769
[ 22.91079788  46.77163103 -11.24995538 ...   3.94064607  22.42362675
  60.03761232] 851028.6502294765
[ 22.91079788  46.77163103 -11.24995538 ...   3.94064607  22.42362675
  60.03761232] 851028.6502294769
[ 22.91079788  46.77163103 -11.24995538 ...   3.94064607  22.42362675
  60.03761232] 851028.6502294769


In [264]:
x = [x_1, x_2, x_3['x'], x_4]
name = ['Pseudoinverse', 'Direct', 'L-BFGS', 'Newton']
for a,b in zip(x,name):
    print(f(a),b,sep='\t')

851028.6502294769	Pseudoinverse
851028.6502294769	Direct
851028.6502294769	L-BFGS
851028.6502294769	Newton


In [204]:
def krylov_newton(f, start, k=10, maxiter=1):
    x    = start
    print(x, f(x))
    # Fuori dal ciclo dato che costanti
    _, V = linalg.eigsh(f_2(x), k, which='LM')
    V    = V.T
    H    = V @ f_2(x) @ V.T
    B    = np.linalg.inv(H)
    for _ in range(maxiter):
        #f_h  = lambda alpha: f(x + V@alpha)
        g    = V @ f_1(x)
        d    = - B @ g @ V
        # """Line Search"""
        ls   = [0.1, 0.3, 0.9]
        l    = np.argmin([f(x+alpha*d) for alpha in ls])
        x    = x + ls[l]*d
        print(l, f(x))

In [205]:
print(n)

1765


In [207]:
krylov_newton(f, start, k=1300, maxiter=15)

[0.85254152 0.89712139 0.63491321 ... 0.71464233 0.35736846 0.27973222] 2073689.4536952334
2 1081179.342951762
2 1071254.2418443286
2 1071154.9908332543
2 1071153.9983231437
2 1071153.9883980418
2 1071153.988298791
2 1071153.988297799
2 1071153.9882977896
1 1071153.9882977887
1 1071153.9882977887
2 1071153.9882977884
0 1071153.988297789
1 1071153.9882977887
1 1071153.9882977884
0 1071153.9882977884


In [372]:
def lanczos(g, d, n, k):
    V    = np.zeros((k+1,n))
    alpha = np.zeros((k,))
    beta = np.zeros((k+1,))
    w    = np.zeros((k,n))
    V[1] = g / np.linalg.norm(g)
    for i in range(1,k):
        if i == k-1:
            w[i] = d
        else:
            w[i] = V[i] @ f_2(x)
        alpha[i] = w[i] @ V[i]
        w[i] = w[i] - alpha[i] * V[i] - beta[i] * V[i-1]
        beta[i+1] = np.linalg.norm(w[i])
        V[i+1] = w[i] / np.linalg.norm(w[i])
    return V[1:]

In [373]:
def dauphin(f, start, k, maxiter=15):
    x = start
    n = start.shape[0]
    d = np.zeros(n)
    print(0, f(x))
    for _ in range(maxiter):
        V    = lanczos(-f_1(x), d, n, k)
        H    = V @ f_2(x) @ V.T
        f_h  = lambda alpha: f(x + alpha@V)
        g    = - V @ f_1(x)
        d    = lambda l: g @ np.linalg.inv(H + l*np.eye(k))
        ls   = [10e-1, 10e-2, 10e-3, 10e-4]
        l    = ls[np.argmin([f_h(d(l)) for l in ls])]
        d    = d(l) @ V
        x    = x + d
        print(l, f(x))

In [374]:
dauphin(f, start, k=16, maxiter=10)

0 1988312.3914933947
0.001 851029.3371762796
0.001 851028.6502296486
0.01 851028.6502294769
1.0 851028.6502294769
0.001 851028.6502294765
1.0 851028.6502294765
0.1 851028.6502294765
0.1 851028.6502294765
1.0 851028.6502294769
1.0 851028.6502294769


In [301]:
dauphin(f, start, k=2, maxiter=20)

0 1988312.3914933947
1592968.9226246432
1547361.2885843161
1541256.9300806383
1539108.6840045631
1537895.0493410516
1536890.147100027
1536016.7396551392
1535213.3655596918
1534461.2502924802
1533738.1052153772
1533036.7729064203
1532347.905068905
1531668.6032894535
1530994.9259260062
1530325.674149038
1529659.1845181738
1528994.9541074522
1528332.2767107408
1527670.9388590925
1527010.637915026
